In [ ]:
# === CELL 1: SETUP ===
!pip install -q yfinance ta

import numpy as np
import pandas as pd
import yfinance as yf
import json
import time
import random
from datetime import datetime, timedelta
from copy import deepcopy
import warnings
warnings.filterwarnings('ignore')

print("🔮 Forecast Evolution Optimizer Ready")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M')}")

In [ ]:
# === CELL 2: R&D DISCOVERY MODE - FORECASTER EVOLUTION ===
# 🔬 EXPERIMENTAL - Let the AI discover new forecasting approaches!

EVOLUTION_CONFIG = {
    'population_size': 100,     # Big population for diversity
    'generations': 60,          # Long exploration
    'mutation_rate': 0.5,       # HIGH mutation - try wild ideas!
    'crossover_rate': 0.5,      # Balance
    'elite_keep': 2,            # Force exploration
    'tournament_size': 3,       # More randomness
    'restart_every': 12,        # Inject chaos
    'wild_card_pct': 0.1,       # 10% random each gen
}

# === ULTRA-WIDE FORECASTER PARAMETERS ===
# Let AI discover optimal indicator combinations and thresholds
FORECAST_PARAMS = {
    # === INDICATOR WEIGHTS (which indicators matter?) ===
    'w_rsi': [0.0, 3.0, 1.0],              # Can go to 3x!
    'w_macd': [0.0, 3.0, 1.0],             
    'w_ema_trend': [0.0, 3.0, 1.0],        
    'w_volume': [0.0, 3.0, 0.5],           
    'w_momentum': [0.0, 3.0, 1.0],         
    'w_volatility': [0.0, 3.0, 0.5],       
    
    # NEW: Experimental indicator weights
    'w_rsi_divergence': [0.0, 2.0, 0.5],   # RSI divergence signal
    'w_consolidation': [0.0, 2.0, 0.5],    # Consolidation breakout
    'w_trend_structure': [0.0, 2.0, 0.5],  # Higher highs/lows
    
    # === RSI THRESHOLDS (explore extremes) ===
    'rsi_oversold': [10, 50, 30],          # Maybe 10? Maybe 50?
    'rsi_overbought': [50, 95, 70],        
    'rsi_neutral_low': [25, 55, 40],       
    'rsi_neutral_high': [45, 75, 60],      
    
    # === CONFIDENCE MECHANICS ===
    'base_confidence': [0.3, 0.8, 0.55],   # Starting confidence
    'confidence_boost': [0.0, 0.4, 0.15],  # Boost for strong signals
    'confidence_decay': [0.7, 0.99, 0.92], # Daily decay
    'confidence_floor': [0.2, 0.5, 0.3],   # Minimum confidence
    
    # === FORECAST DRIFT (price movement expectations) ===
    'drift_bull': [0.0005, 0.015, 0.002],  # Daily bullish drift (up to 1.5%!)
    'drift_bear': [-0.015, -0.0005, -0.002], # Daily bearish drift
    'drift_neutral': [-0.002, 0.002, 0.0], # Neutral drift (can be non-zero!)
    'volatility_scale': [0.1, 2.0, 0.5],   # Vol impact
    
    # === SIGNAL TIMING ===
    'decay_start_day': [1, 21, 10],        # When to start decaying
    'max_daily_move': [0.02, 0.15, 0.05],  # Max daily move (2-15%)
    'forecast_horizon': [3, 14, 7],        # Days to forecast
    
    # === MULTI-TIMEFRAME ===
    'ema_short_period': [3, 21, 8],        # Short EMA
    'ema_long_period': [13, 89, 34],       # Long EMA (up to 89!)
    'momentum_lookback': [2, 21, 5],       # Momentum calc
    'rsi_period': [7, 21, 14],             # RSI period itself!
    
    # === SIGNAL THRESHOLDS (when to act) ===
    'signal_buy_threshold': [0.1, 0.8, 0.5],   # Lower = more buys
    'signal_sell_threshold': [-0.8, -0.1, -0.5], # Higher = more sells
    'signal_strong_mult': [1.0, 2.0, 1.3],     # Strong signal multiplier
    
    # === EXPERIMENTAL: ADAPTIVE PARAMETERS ===
    'vol_adjustment': [0.5, 2.0, 1.0],     # Adjust for volatility
    'trend_confirmation': [0.0, 1.0, 0.5], # Require trend alignment
    'mean_reversion_weight': [0.0, 1.0, 0.3], # Mean reversion factor
}

WATCHLIST = [
    # High volatility targets
    'IONQ', 'RGTI', 'QUBT', 'SMR', 'OKLO', 'LEU',
    # Tech
    'NVDA', 'AMD', 'MRVL', 'CRDO', 'MU', 'APLD', 'SERV',
    # Momentum
    'TSLA', 'META', 'GOOGL', 'HOOD', 'SNOW', 'LUNR',
    # Recovery
    'RIVN', 'LYFT', 'UUUU', 'CCJ',
    # Benchmarks
    'SPY', 'QQQ', 'TQQQ'
]

print("🔬 R&D DISCOVERY MODE - FORECASTER EVOLUTION")
print("=" * 60)
print(f"   Population: {EVOLUTION_CONFIG['population_size']}")
print(f"   Generations: {EVOLUTION_CONFIG['generations']}")
print(f"   Mutation Rate: {EVOLUTION_CONFIG['mutation_rate']} (HIGH)")
print(f"   Parameters: {len(FORECAST_PARAMS)} (many experimental)")
print(f"   Tickers: {len(WATCHLIST)}")
print(f"\n🎯 GOAL: Discover better forecasting methods")
print(f"   - Which indicators actually predict direction?")
print(f"   - What RSI thresholds work best?")
print(f"   - How should confidence decay?")
print(f"   - Can we find unconventional combinations?")

In [ ]:
# === CELL 3: LOAD DATA ===
print("📥 Loading data...")

data_dict = {}
for ticker in WATCHLIST:
    try:
        df = yf.download(ticker, period='2y', progress=False)
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        df = df.reset_index()
        for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        if len(df) > 100:
            data_dict[ticker] = df
            print(f"   ✓ {ticker}: {len(df)} days")
    except Exception as e:
        print(f"   ✗ {ticker}: {e}")

# Split 70/30 train/test
train_data = {}
test_data = {}
for ticker, df in data_dict.items():
    split = int(len(df) * 0.7)
    train_data[ticker] = df.iloc[:split].reset_index(drop=True)
    test_data[ticker] = df.iloc[split:].reset_index(drop=True)

print(f"\n✅ Train: {len(train_data[list(train_data.keys())[0]])} days")
print(f"✅ Test: {len(test_data[list(test_data.keys())[0]])} days")

In [ ]:
# === CELL 4: ENHANCED FEATURE ENGINE FOR FORECASTING ===
def compute_forecast_features(df, params=None):
    """Compute features with EXPERIMENTAL additions for forecasting"""
    if params is None:
        params = {name: best for name, (_, _, best) in FORECAST_PARAMS.items()}
    
    df = df.copy()
    c = df['Close'].astype(float)
    h = df['High'].astype(float)
    l = df['Low'].astype(float)
    v = df['Volume'].astype(float)
    
    # === RSI (with evolved period) ===
    rsi_period = int(params.get('rsi_period', 14))
    delta = c.diff()
    gain = delta.where(delta > 0, 0).rolling(rsi_period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(rsi_period).mean()
    df['rsi'] = 100 - (100 / (1 + gain / (loss + 1e-10)))
    
    # Multi-period RSI for divergence detection
    for p in [7, 14, 21]:
        gain_p = delta.where(delta > 0, 0).rolling(p).mean()
        loss_p = (-delta.where(delta < 0, 0)).rolling(p).mean()
        df[f'rsi_{p}'] = 100 - (100 / (1 + gain_p / (loss_p + 1e-10)))
    
    # === EMAs (evolved periods) ===
    short_ema = int(params.get('ema_short_period', 8))
    long_ema = int(params.get('ema_long_period', 34))
    df['ema_short'] = c.ewm(span=short_ema).mean()
    df['ema_long'] = c.ewm(span=long_ema).mean()
    
    # Standard EMAs for ribbon
    for p in [5, 8, 13, 21, 34, 55, 89]:
        df[f'ema_{p}'] = c.ewm(span=p).mean()
    
    # EMA trend signal
    df['ema_bullish'] = (df['ema_short'] > df['ema_long']).astype(float)
    df['ema_spread'] = (df['ema_short'] - df['ema_long']) / c * 100
    df['ema_spread_expanding'] = (df['ema_spread'].abs() > df['ema_spread'].abs().shift(3)).astype(float)
    
    # === MACD (standard + custom) ===
    df['macd'] = c.ewm(span=12).mean() - c.ewm(span=26).mean()
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    df['macd_bullish'] = (df['macd'] > df['macd_signal']).astype(float)
    df['macd_hist_rising'] = (df['macd_hist'] > df['macd_hist'].shift(1)).astype(float)
    
    # Custom MACD periods
    df['macd_fast'] = c.ewm(span=5).mean() - c.ewm(span=13).mean()
    df['macd_fast_signal'] = df['macd_fast'].ewm(span=5).mean()
    
    # === Momentum (evolved lookback) ===
    mom_days = int(params.get('momentum_lookback', 5))
    df['momentum'] = c.pct_change(mom_days) * 100
    df['momentum_5d'] = c.pct_change(5) * 100
    df['momentum_10d'] = c.pct_change(10) * 100
    df['momentum_21d'] = c.pct_change(21) * 100
    
    # Momentum acceleration
    df['mom_accel'] = df['momentum'] - df['momentum'].shift(3)
    df['mom_accel_pos'] = (df['mom_accel'] > 0).astype(float)
    
    # === Volatility ===
    tr1 = h - l
    tr2 = abs(h - c.shift(1))
    tr3 = abs(l - c.shift(1))
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    df['atr'] = tr.rolling(14).mean()
    df['atr_pct'] = df['atr'] / c * 100
    df['volatility'] = c.pct_change().rolling(20).std() * np.sqrt(252) * 100
    df['vol_expanding'] = (df['atr_pct'] > df['atr_pct'].rolling(20).mean()).astype(float)
    df['vol_squeeze'] = (df['atr_pct'] < df['atr_pct'].rolling(50).mean() * 0.7).astype(float)
    
    # === Volume ===
    df['vol_sma'] = v.rolling(20).mean()
    df['vol_ratio'] = v / (df['vol_sma'] + 1)
    df['vol_spike'] = (df['vol_ratio'] > 2).astype(float)
    df['vol_trend'] = v.rolling(5).mean() / (v.rolling(20).mean() + 1)
    
    # OBV (On Balance Volume)
    df['obv'] = (np.sign(c.diff()) * v).cumsum()
    df['obv_trend'] = np.sign(df['obv'].diff(5))
    
    # === Price Position ===
    df['high_20d'] = h.rolling(20).max()
    df['low_20d'] = l.rolling(20).min()
    df['price_position'] = (c - df['low_20d']) / (df['high_20d'] - df['low_20d'] + 1e-10)
    
    df['high_52w'] = h.rolling(252).max()
    df['low_52w'] = l.rolling(252).min()
    df['price_position_52w'] = (c - df['low_52w']) / (df['high_52w'] - df['low_52w'] + 1e-10)
    
    # === Trend Strength ===
    df['trend_5d'] = np.sign(c.diff(5))
    df['trend_10d'] = np.sign(c.diff(10))
    df['trend_21d'] = np.sign(c.diff(21))
    df['trend_align'] = (df['trend_5d'] + df['trend_10d'] + df['trend_21d']) / 3
    
    # === EXPERIMENTAL FEATURES ===
    
    # RSI Divergence
    df['price_low_5d'] = c.rolling(5).min()
    df['rsi_at_low'] = df['rsi'].rolling(5).min()
    df['bullish_divergence'] = ((c <= df['price_low_5d'] * 1.02) & 
                                 (df['rsi'] > df['rsi_at_low'] + 5)).astype(float)
    
    df['price_high_5d'] = c.rolling(5).max()
    df['rsi_at_high'] = df['rsi'].rolling(5).max()
    df['bearish_divergence'] = ((c >= df['price_high_5d'] * 0.98) & 
                                 (df['rsi'] < df['rsi_at_high'] - 5)).astype(float)
    
    # Consolidation
    df['range_10d'] = (h.rolling(10).max() - l.rolling(10).min()) / c * 100
    df['consolidating'] = (df['range_10d'] < df['range_10d'].rolling(30).mean() * 0.6).astype(float)
    
    # Higher Highs / Higher Lows
    df['higher_low'] = (l > l.shift(5)).astype(float)
    df['higher_high'] = (h > h.shift(5)).astype(float)
    df['uptrend_structure'] = (df['higher_low'] + df['higher_high']) / 2
    df['lower_low'] = (l < l.shift(5)).astype(float)
    df['lower_high'] = (h < h.shift(5)).astype(float)
    df['downtrend_structure'] = (df['lower_low'] + df['lower_high']) / 2
    
    # Support/Resistance proximity
    df['near_support'] = (c < df['low_20d'] * 1.03).astype(float)
    df['near_resistance'] = (c > df['high_20d'] * 0.97).astype(float)
    
    # Mean Reversion potential
    df['distance_from_ema21'] = (c / df['ema_21'] - 1) * 100
    df['overextended_up'] = (df['distance_from_ema21'] > 10).astype(float)
    df['overextended_down'] = (df['distance_from_ema21'] < -10).astype(float)
    
    df = df.replace([np.inf, -np.inf], np.nan).ffill().bfill().fillna(0)
    return df

print("🧠 Computing ENHANCED forecast features...")
train_features = {t: compute_forecast_features(df) for t, df in train_data.items()}
test_features = {t: compute_forecast_features(df) for t, df in test_data.items()}
print(f"✅ Features ready - {len(train_features[list(train_features.keys())[0]].columns)} total features")

In [ ]:
# === CELL 5: FORECAST DNA CLASS ===

class ForecastDNA:
    """A forecasting strategy's genetic code"""
    
    def __init__(self, params=None):
        if params:
            self.params = params
        else:
            # Random initialization
            self.params = {}
            for name, (min_v, max_v, _) in FORECAST_PARAMS.items():
                if isinstance(min_v, int) and isinstance(max_v, int):
                    self.params[name] = random.randint(min_v, max_v)
                else:
                    self.params[name] = random.uniform(min_v, max_v)
        
        self.fitness = 0
        self.accuracy = 0
        self.direction_accuracy = 0
    
    def mutate(self):
        """Randomly mutate one parameter"""
        param = random.choice(list(self.params.keys()))
        min_v, max_v, _ = FORECAST_PARAMS[param]
        
        # Mutation: adjust by 10-30%
        current = self.params[param]
        range_size = max_v - min_v
        mutation = random.uniform(-0.3, 0.3) * range_size
        new_val = current + mutation
        
        # Clamp to bounds
        new_val = max(min_v, min(max_v, new_val))
        
        if isinstance(min_v, int) and isinstance(max_v, int):
            new_val = int(new_val)
        
        self.params[param] = new_val
    
    @staticmethod
    def crossover(parent1, parent2):
        """Breed two forecasters"""
        child_params = {}
        for param in parent1.params:
            if random.random() < 0.5:
                child_params[param] = parent1.params[param]
            else:
                child_params[param] = parent2.params[param]
            
            # 20% chance to blend
            if random.random() < 0.2:
                blend = (parent1.params[param] + parent2.params[param]) / 2
                min_v, max_v, _ = FORECAST_PARAMS[param]
                if isinstance(min_v, int) and isinstance(max_v, int):
                    blend = int(blend)
                child_params[param] = blend
        
        return ForecastDNA(child_params)
    
    def __repr__(self):
        return f"ForecastDNA(fitness={self.fitness:.1f}, acc={self.accuracy:.1f}%)"

print("✅ ForecastDNA class ready")
# Test
test_dna = ForecastDNA()
print(f"   Sample DNA params: {len(test_dna.params)} genes")

In [ ]:
# === CELL 6: EXPERIMENTAL EVOLVED FORECASTER ===

class EvolvedForecaster:
    """
    Forecaster using evolved parameters.
    EXPERIMENTAL - includes unconventional signal combinations.
    """
    
    def __init__(self, dna):
        self.p = dna.params if hasattr(dna, 'params') else dna
    
    def compute_signal_score(self, row):
        """
        Compute weighted signal score from ALL indicators.
        Returns: score from -1 (very bearish) to +1 (very bullish)
        """
        score = 0
        total_weight = 0
        signals = {}  # Track individual signals for analysis
        
        # === RSI Signal ===
        rsi = row.get('rsi', 50)
        rsi_signal = 0
        if rsi < self.p['rsi_oversold']:
            rsi_signal = 1.0  # Oversold = bullish
        elif rsi > self.p['rsi_overbought']:
            rsi_signal = -1.0  # Overbought = bearish
        elif rsi < self.p['rsi_neutral_low']:
            rsi_signal = 0.5
        elif rsi > self.p['rsi_neutral_high']:
            rsi_signal = -0.5
        else:
            # Gradient in neutral zone
            rsi_signal = (50 - rsi) / 50 * 0.3
        
        signals['rsi'] = rsi_signal
        score += rsi_signal * self.p['w_rsi']
        total_weight += self.p['w_rsi']
        
        # === MACD Signal ===
        macd_bullish = row.get('macd_bullish', 0.5)
        macd_hist = row.get('macd_hist', 0)
        macd_hist_rising = row.get('macd_hist_rising', 0.5)
        
        macd_signal = (macd_bullish - 0.5) * 2
        if macd_hist > 0 and macd_hist_rising > 0:
            macd_signal = min(1.0, macd_signal + 0.4)
        elif macd_hist < 0 and macd_hist_rising < 1:
            macd_signal = max(-1.0, macd_signal - 0.4)
        
        signals['macd'] = macd_signal
        score += macd_signal * self.p['w_macd']
        total_weight += self.p['w_macd']
        
        # === EMA Trend Signal ===
        ema_bullish = row.get('ema_bullish', 0.5)
        ema_spread = row.get('ema_spread', 0)
        ema_expanding = row.get('ema_spread_expanding', 0)
        
        ema_signal = (ema_bullish - 0.5) * 2
        # Amplify if spread is large and expanding
        if abs(ema_spread) > 2 and ema_expanding > 0:
            ema_signal *= 1.5
        ema_signal = np.clip(ema_signal, -1, 1)
        
        signals['ema'] = ema_signal
        score += ema_signal * self.p['w_ema_trend']
        total_weight += self.p['w_ema_trend']
        
        # === Volume Signal ===
        vol_ratio = row.get('vol_ratio', 1)
        vol_trend = row.get('vol_trend', 1)
        obv_trend = row.get('obv_trend', 0)
        momentum = row.get('momentum', 0)
        
        if vol_ratio > 1.5:
            vol_signal = np.sign(momentum) * min(1.0, (vol_ratio - 1) / 2)
            # OBV confirmation
            if obv_trend == np.sign(momentum):
                vol_signal *= 1.3
        else:
            vol_signal = obv_trend * 0.3
        
        signals['volume'] = vol_signal
        score += vol_signal * self.p['w_volume']
        total_weight += self.p['w_volume']
        
        # === Momentum Signal ===
        mom_5d = row.get('momentum_5d', 0)
        mom_10d = row.get('momentum_10d', 0)
        mom_accel = row.get('mom_accel', 0)
        
        # Multi-timeframe momentum
        mom_signal = np.clip((mom_5d + mom_10d * 0.5) / 15, -1, 1)
        # Acceleration bonus
        if mom_accel > 0 and mom_signal > 0:
            mom_signal = min(1.0, mom_signal * 1.3)
        elif mom_accel < 0 and mom_signal < 0:
            mom_signal = max(-1.0, mom_signal * 1.3)
        
        signals['momentum'] = mom_signal
        score += mom_signal * self.p['w_momentum']
        total_weight += self.p['w_momentum']
        
        # === Volatility Signal ===
        atr_pct = row.get('atr_pct', 2)
        vol_squeeze = row.get('vol_squeeze', 0)
        vol_expanding = row.get('vol_expanding', 0)
        
        # High vol = uncertainty, low vol = potential breakout
        vol_adj = self.p.get('vol_adjustment', 1.0)
        if vol_squeeze > 0:
            vol_signal = 0.3 * np.sign(momentum)  # Potential breakout
        elif vol_expanding > 0 and atr_pct > 4:
            vol_signal = -0.2  # High uncertainty
        else:
            vol_signal = 0
        
        signals['volatility'] = vol_signal
        score += vol_signal * self.p['w_volatility']
        total_weight += self.p['w_volatility']
        
        # === EXPERIMENTAL SIGNALS ===
        
        # RSI Divergence
        bull_div = row.get('bullish_divergence', 0)
        bear_div = row.get('bearish_divergence', 0)
        div_signal = bull_div - bear_div
        signals['divergence'] = div_signal
        w_div = self.p.get('w_rsi_divergence', 0.5)
        score += div_signal * w_div
        total_weight += w_div
        
        # Consolidation Breakout
        consolidating = row.get('consolidating', 0)
        consol_signal = consolidating * np.sign(momentum) * 0.5
        signals['consolidation'] = consol_signal
        w_consol = self.p.get('w_consolidation', 0.5)
        score += consol_signal * w_consol
        total_weight += w_consol
        
        # Trend Structure
        uptrend = row.get('uptrend_structure', 0)
        downtrend = row.get('downtrend_structure', 0)
        structure_signal = uptrend - downtrend
        signals['structure'] = structure_signal
        w_struct = self.p.get('w_trend_structure', 0.5)
        score += structure_signal * w_struct
        total_weight += w_struct
        
        # Mean Reversion
        dist_from_ema = row.get('distance_from_ema21', 0)
        mr_weight = self.p.get('mean_reversion_weight', 0.3)
        if abs(dist_from_ema) > 8:
            mr_signal = -np.sign(dist_from_ema) * min(1.0, abs(dist_from_ema) / 15)
            score += mr_signal * mr_weight
            total_weight += mr_weight
            signals['mean_reversion'] = mr_signal
        
        # Normalize score
        if total_weight > 0:
            score = score / total_weight
        
        # Trend confirmation requirement
        trend_conf = self.p.get('trend_confirmation', 0.5)
        trend_align = row.get('trend_align', 0)
        if trend_conf > 0.5 and np.sign(score) != np.sign(trend_align) and abs(trend_align) > 0.3:
            score *= (1 - trend_conf * 0.5)  # Reduce confidence if against trend
        
        return np.clip(score, -1, 1), signals
    
    def generate_signal(self, row):
        """Generate BUY/SELL/HOLD signal with confidence."""
        score, signals = self.compute_signal_score(row)
        
        strong_mult = self.p.get('signal_strong_mult', 1.3)
        conf_floor = self.p.get('confidence_floor', 0.3)
        
        if score > self.p['signal_buy_threshold']:
            signal = 'BUY'
            # Strong signal bonus
            if score > self.p['signal_buy_threshold'] * strong_mult:
                confidence = self.p['base_confidence'] + self.p['confidence_boost'] * 1.5
            else:
                confidence = self.p['base_confidence'] + self.p['confidence_boost'] * abs(score)
        elif score < self.p['signal_sell_threshold']:
            signal = 'SELL'
            if score < self.p['signal_sell_threshold'] * strong_mult:
                confidence = self.p['base_confidence'] + self.p['confidence_boost'] * 1.5
            else:
                confidence = self.p['base_confidence'] + self.p['confidence_boost'] * abs(score)
        else:
            signal = 'HOLD'
            confidence = self.p['base_confidence']
        
        confidence = max(conf_floor, min(0.95, confidence))
        
        return signal, confidence, score, signals
    
    def forecast_price(self, current_price, signal, confidence, day, volatility):
        """Generate price forecast for a specific day."""
        # Decay confidence over time
        decay_start = int(self.p.get('decay_start_day', 10))
        decay_factor = 1.0
        if day > decay_start:
            decay_factor = self.p['confidence_decay'] ** (day - decay_start)
        
        conf_floor = self.p.get('confidence_floor', 0.3)
        effective_conf = max(conf_floor, confidence * decay_factor)
        
        # Determine drift
        if signal == 'BUY':
            drift = self.p['drift_bull'] * effective_conf
        elif signal == 'SELL':
            drift = self.p['drift_bear'] * effective_conf
        else:
            drift = self.p.get('drift_neutral', 0)
        
        # Volatility adjustment
        vol_adj = self.p.get('vol_adjustment', 1.0)
        vol_scale = self.p['volatility_scale'] * vol_adj
        random_component = np.random.normal(0, volatility * vol_scale / 100)
        
        # Price change
        price_change = drift + random_component
        price_change = np.clip(price_change, -self.p['max_daily_move'], self.p['max_daily_move'])
        
        new_price = current_price * (1 + price_change)
        
        return new_price, effective_conf

print("✅ EXPERIMENTAL EvolvedForecaster class ready")
print("   - Multi-indicator fusion")
print("   - Divergence detection")
print("   - Mean reversion overlay")
print("   - Trend confirmation")

In [ ]:
# === CELL 7: FITNESS EVALUATOR ===

def evaluate_forecast_fitness(dna, features_dict, data_dict, forecast_days=7):
    """
    Evaluate forecaster accuracy.
    
    Fitness based on:
    - Direction accuracy (did we predict up/down correctly?)
    - Magnitude accuracy (how close was our forecast?)
    - Signal quality (did BUY signals lead to gains?)
    """
    forecaster = EvolvedForecaster(dna)
    
    direction_correct = 0
    direction_total = 0
    magnitude_errors = []
    signal_returns = {'BUY': [], 'SELL': [], 'HOLD': []}
    
    tickers = list(features_dict.keys())
    
    for ticker in tickers:
        df = features_dict[ticker]
        data = data_dict[ticker]
        
        # Test on multiple points
        test_points = range(60, len(df) - forecast_days - 1, 5)  # Every 5 days
        
        for start_idx in test_points:
            try:
                row = df.iloc[start_idx].to_dict()
                current_price = float(data['Close'].iloc[start_idx])
                
                # Generate signal
                signal, confidence, score = forecaster.generate_signal(row)
                
                # Get actual future prices
                future_prices = []
                for d in range(1, forecast_days + 1):
                    if start_idx + d < len(data):
                        future_prices.append(float(data['Close'].iloc[start_idx + d]))
                
                if len(future_prices) < forecast_days:
                    continue
                
                # Actual return over forecast period
                actual_return = (future_prices[-1] / current_price - 1) * 100
                
                # Direction accuracy
                predicted_direction = 1 if signal == 'BUY' else (-1 if signal == 'SELL' else 0)
                actual_direction = 1 if actual_return > 1 else (-1 if actual_return < -1 else 0)
                
                if predicted_direction != 0:  # Only count non-HOLD predictions
                    direction_total += 1
                    if predicted_direction == actual_direction:
                        direction_correct += 1
                
                # Signal quality (did signal lead to profit?)
                signal_returns[signal].append(actual_return)
                
                # Generate forecast and measure error
                volatility = float(row.get('atr_pct', 2))
                forecasted_price = current_price
                
                for d in range(forecast_days):
                    forecasted_price, _ = forecaster.forecast_price(
                        forecasted_price, signal, confidence, d + 1, volatility
                    )
                
                forecast_return = (forecasted_price / current_price - 1) * 100
                magnitude_error = abs(forecast_return - actual_return)
                magnitude_errors.append(magnitude_error)
                
            except Exception as e:
                continue
    
    # Calculate metrics
    direction_accuracy = (direction_correct / max(direction_total, 1)) * 100
    avg_magnitude_error = np.mean(magnitude_errors) if magnitude_errors else 100
    
    # Signal quality scores
    buy_avg = np.mean(signal_returns['BUY']) if signal_returns['BUY'] else 0
    sell_avg = np.mean(signal_returns['SELL']) if signal_returns['SELL'] else 0
    
    # Good BUY signals should have positive returns
    # Good SELL signals should have negative returns (or we avoided losses)
    buy_quality = max(0, buy_avg)  # Reward positive returns on BUY
    sell_quality = max(0, -sell_avg)  # Reward avoided losses on SELL
    signal_quality = buy_quality + sell_quality
    
    # FITNESS FORMULA:
    # High direction accuracy + low magnitude error + good signal quality
    fitness = (
        direction_accuracy * 2 +              # Weight direction accuracy heavily
        max(0, 50 - avg_magnitude_error) +    # Penalty for large errors
        signal_quality * 10                    # Reward good signals
    )
    
    return fitness, direction_accuracy, avg_magnitude_error, buy_avg, sell_avg

print("✅ Fitness evaluator ready")

In [ ]:
# === CELL 8: EVOLUTION ENGINE ===

def evolve_forecasters(population, features, data):
    """One generation of evolution"""
    
    # Evaluate all
    for dna in population:
        dna.fitness, dna.accuracy, _, _, _ = evaluate_forecast_fitness(dna, features, data)
    
    # Sort by fitness
    population.sort(key=lambda x: x.fitness, reverse=True)
    
    # Keep elite
    new_pop = population[:EVOLUTION_CONFIG['elite_keep']]
    
    # Fill rest with offspring
    while len(new_pop) < EVOLUTION_CONFIG['population_size']:
        # Tournament selection
        def tournament():
            contestants = random.sample(population[:25], EVOLUTION_CONFIG['tournament_size'])
            return max(contestants, key=lambda x: x.fitness)
        
        parent1 = tournament()
        parent2 = tournament()
        
        # Crossover
        if random.random() < EVOLUTION_CONFIG['crossover_rate']:
            child = ForecastDNA.crossover(parent1, parent2)
        else:
            child = ForecastDNA(deepcopy(parent1.params))
        
        # Mutation
        if random.random() < EVOLUTION_CONFIG['mutation_rate']:
            child.mutate()
        
        new_pop.append(child)
    
    return new_pop

print("✅ Evolution engine ready")

In [ ]:
# === CELL 9: R&D EVOLUTION RUN ===
print("=" * 70)
print("🔬 R&D DISCOVERY MODE - FORECASTER EVOLUTION")
print("=" * 70)

# Initialize population
population = [ForecastDNA() for _ in range(EVOLUTION_CONFIG['population_size'])]

# Seed with human baseline
human_params = {name: best for name, (_, _, best) in FORECAST_PARAMS.items()}
population[0] = ForecastDNA(human_params)

# Seed extreme variants
rsi_heavy = human_params.copy()
rsi_heavy['w_rsi'] = 2.5
rsi_heavy['w_macd'] = 0.5
rsi_heavy['w_momentum'] = 0.5
population[1] = ForecastDNA(rsi_heavy)

momentum_heavy = human_params.copy()
momentum_heavy['w_momentum'] = 2.5
momentum_heavy['w_rsi'] = 0.5
momentum_heavy['w_macd'] = 0.5
population[2] = ForecastDNA(momentum_heavy)

trend_heavy = human_params.copy()
trend_heavy['w_ema_trend'] = 2.5
trend_heavy['w_trend_structure'] = 1.5
population[3] = ForecastDNA(trend_heavy)

mean_reversion = human_params.copy()
mean_reversion['mean_reversion_weight'] = 0.8
mean_reversion['signal_buy_threshold'] = 0.3
population[4] = ForecastDNA(mean_reversion)

best_ever = None
history = []
discoveries = []

start_time = time.time()

print(f"\n🚀 Starting with {EVOLUTION_CONFIG['population_size']} forecasters")
print(f"   Seeded: Human, RSI-Heavy, Momentum-Heavy, Trend-Heavy, Mean-Reversion\n")

for gen in range(EVOLUTION_CONFIG['generations']):
    # Evolve on TRAINING data
    population = evolve_forecasters(population, train_features, train_data)
    
    best = population[0]
    avg_fitness = np.mean([d.fitness for d in population[:10]])
    diversity = np.std([d.fitness for d in population])
    
    # Track best ever
    if best_ever is None or best.fitness > best_ever.fitness:
        best_ever = ForecastDNA(deepcopy(best.params))
        best_ever.fitness = best.fitness
        best_ever.accuracy = best.accuracy
        
        discoveries.append({
            'gen': gen,
            'fitness': best.fitness,
            'accuracy': best.accuracy,
            'params': best.params.copy()
        })
        marker = "🆕 NEW BEST!"
    else:
        marker = ""
    
    history.append({
        'gen': gen,
        'best_fitness': best.fitness,
        'best_accuracy': best.accuracy,
        'avg_fitness': avg_fitness,
        'diversity': diversity
    })
    
    # Chaos injection
    restart_every = EVOLUTION_CONFIG.get('restart_every', 12)
    if gen > 0 and gen % restart_every == 0:
        num_replace = int(EVOLUTION_CONFIG['population_size'] * 0.25)
        for i in range(-num_replace, 0):
            population[i] = ForecastDNA()
        print(f"   🔄 Gen {gen}: Injected {num_replace} wild cards")
    
    # Wild cards each generation
    wild_pct = EVOLUTION_CONFIG.get('wild_card_pct', 0.1)
    num_wild = max(1, int(EVOLUTION_CONFIG['population_size'] * wild_pct))
    for i in range(num_wild):
        idx = random.randint(EVOLUTION_CONFIG['elite_keep'], len(population) - 1)
        population[idx] = ForecastDNA()
    
    if gen % 5 == 0:
        elapsed = time.time() - start_time
        print(f"Gen {gen:3d} | Fit: {best.fitness:8.1f} | Acc: {best.accuracy:5.1f}% | Avg: {avg_fitness:7.1f} | {elapsed:.0f}s {marker}")

print("-" * 70)
print(f"\n🏆 R&D COMPLETE in {time.time()-start_time:.0f}s")
print(f"   Best Fitness: {best_ever.fitness:.1f}")
print(f"   Best Accuracy: {best_ever.accuracy:.1f}%")
print(f"   Discoveries: {len(discoveries)}")

In [ ]:
# === CELL 10: TEST ON UNSEEN DATA ===
print("=" * 70)
print("🧪 TESTING EVOLVED FORECASTER ON UNSEEN DATA")
print("=" * 70)

# Test evolved best
evolved_fit, evolved_acc, evolved_err, evolved_buy, evolved_sell = evaluate_forecast_fitness(
    best_ever, test_features, test_data
)

# Test human baseline
human_dna = ForecastDNA(human_params)
human_fit, human_acc, human_err, human_buy, human_sell = evaluate_forecast_fitness(
    human_dna, test_features, test_data
)

print(f"\n{'Metric':<25} {'Human':>15} {'Evolved':>15} {'Winner':>10}")
print("-" * 65)
print(f"{'Direction Accuracy':<25} {human_acc:>14.1f}% {evolved_acc:>14.1f}% {'🧬' if evolved_acc > human_acc else '👤'}")
print(f"{'Magnitude Error':<25} {human_err:>14.1f}% {evolved_err:>14.1f}% {'🧬' if evolved_err < human_err else '👤'}")
print(f"{'BUY Signal Avg Return':<25} {human_buy:>+14.2f}% {evolved_buy:>+14.2f}% {'🧬' if evolved_buy > human_buy else '👤'}")
print(f"{'SELL Signal Avg Return':<25} {human_sell:>+14.2f}% {evolved_sell:>+14.2f}% {'🧬' if evolved_sell < human_sell else '👤'}")
print(f"{'Fitness Score':<25} {human_fit:>15.1f} {evolved_fit:>15.1f} {'🧬' if evolved_fit > human_fit else '👤'}")

improvement = evolved_acc - human_acc
print(f"\n📈 Direction accuracy improved by {improvement:+.1f} percentage points")

# Show discoveries
print(f"\n🔬 R&D DISCOVERIES ({len(discoveries)} breakthroughs):")
for d in discoveries[:5]:
    print(f"   Gen {d['gen']:3d}: Fitness {d['fitness']:.1f}, Accuracy {d['accuracy']:.1f}%")

In [ ]:
# === CELL 11: SHOW EVOLVED PARAMETERS ===
print("=" * 70)
print("🧬 EVOLVED OPTIMAL FORECASTER PARAMETERS")
print("=" * 70)

print(f"\n{'Parameter':<30} {'Human':>12} {'Evolved':>12} {'Change':>12}")
print("-" * 70)

for param in FORECAST_PARAMS:
    human_val = human_params[param]
    evolved_val = best_ever.params[param]
    
    if isinstance(human_val, float):
        if human_val != 0:
            change = (evolved_val - human_val) / abs(human_val) * 100
        else:
            change = evolved_val * 100
        print(f"{param:<30} {human_val:>12.3f} {evolved_val:>12.3f} {change:>+11.1f}%")
    else:
        change = evolved_val - human_val
        print(f"{param:<30} {human_val:>12} {evolved_val:>12} {change:>+12}")

# Group by category
print("\n" + "=" * 70)
print("📋 KEY INSIGHTS FROM R&D EVOLUTION:")
print("=" * 70)

# Classic indicator weights
print("\n🎚️ CLASSIC INDICATOR WEIGHTS:")
classic_weights = ['w_rsi', 'w_macd', 'w_ema_trend', 'w_volume', 'w_momentum', 'w_volatility']
for name in classic_weights:
    val = best_ever.params[name]
    indicator = name.replace('w_', '').upper()
    bar = '█' * int(val * 8)
    print(f"   {indicator:<15} {bar:<20} {val:.2f}")

# Experimental indicator weights
print("\n🧪 EXPERIMENTAL INDICATOR WEIGHTS:")
exp_weights = ['w_rsi_divergence', 'w_consolidation', 'w_trend_structure', 'w_mean_reversion']
for name in exp_weights:
    val = best_ever.params.get(name, 0)
    indicator = name.replace('w_', '').upper()
    bar = '█' * int(val * 8)
    status = "🔥" if val > 1.0 else "✅" if val > 0.5 else "❓"
    print(f"   {indicator:<18} {bar:<16} {val:.2f} {status}")

# Thresholds
print("\n🎯 EVOLVED THRESHOLDS:")
print(f"   RSI Oversold: < {best_ever.params['rsi_oversold']:.0f}")
print(f"   RSI Overbought: > {best_ever.params['rsi_overbought']:.0f}")
print(f"   Signal Buy Threshold: {best_ever.params['signal_buy_threshold']:.2f}")
print(f"   Signal Sell Threshold: {best_ever.params['signal_sell_threshold']:.2f}")

# Forecast mechanics
print("\n⚙️ FORECAST MECHANICS:")
print(f"   Daily Bull Drift: {best_ever.params['drift_bull']*100:.2f}%")
print(f"   Daily Bear Drift: {best_ever.params['drift_bear']*100:.2f}%")
print(f"   Max Daily Move: {best_ever.params['max_daily_move']*100:.1f}%")
print(f"   Confidence Decay: {best_ever.params['confidence_decay']:.2f} per day")
print(f"   Decay Start Day: {int(best_ever.params['decay_start_day'])}")

# New R&D parameters
print("\n🔬 R&D DISCOVERIES:")
print(f"   Mean Reversion Weight: {best_ever.params.get('mean_reversion_weight', 0):.2f}")
print(f"   Trend Continuation Bias: {best_ever.params.get('trend_continuation_bias', 0):.2f}")
print(f"   Signal Smoothing: {best_ever.params.get('signal_smoothing', 0):.2f}")
print(f"   Confidence Floor: {best_ever.params.get('confidence_floor', 0):.2f}")

In [ ]:
# === CELL 12: LIVE FORECAST DEMO ===
print("=" * 70)
print("🔮 LIVE FORECAST DEMO (Using Evolved Parameters)")
print("=" * 70)

# Pick top quantum/AI tickers to forecast
demo_tickers = ['RGTI', 'IONQ', 'SMR', 'OKLO', 'TSLA', 'NVDA', 'AMD', 'PLTR'][:8]

evolved_forecaster = EvolvedForecaster(best_ever)

print(f"\n{'Ticker':<8} {'Price':>10} {'Signal':>8} {'Conf':>8} {'Score':>8} {'7d Fcst':>10} {'Expected':>10}")
print("-" * 75)

forecasts = []
for ticker in demo_tickers:
    if ticker not in test_features:
        continue
    
    df = test_features[ticker]
    data = test_data[ticker]
    
    row = df.iloc[-1].to_dict()
    current_price = float(data['Close'].iloc[-1])
    volatility = row.get('atr_pct', 2)
    
    signal, confidence, score = evolved_forecaster.generate_signal(row)
    
    # 7-day forecast
    forecast_price = current_price
    for d in range(7):
        forecast_price, _ = evolved_forecaster.forecast_price(
            forecast_price, signal, confidence, d + 1, volatility
        )
    
    expected_return = (forecast_price / current_price - 1) * 100
    
    signal_emoji = '🟢' if signal == 'BUY' else '🔴' if signal == 'SELL' else '🟡'
    
    print(f"{ticker:<8} ${current_price:>9.2f} {signal_emoji} {signal:<6} {confidence*100:>6.0f}% {score:>+7.2f} ${forecast_price:>9.2f} {expected_return:>+9.1f}%")
    
    forecasts.append({
        'ticker': ticker,
        'current_price': current_price,
        'signal': signal,
        'confidence': confidence,
        'raw_score': score,
        'forecast_7d': forecast_price,
        'expected_return': expected_return
    })

print("\n" + "-" * 75)
buys = [f for f in forecasts if f['signal'] == 'BUY']
sells = [f for f in forecasts if f['signal'] == 'SELL']
holds = len(forecasts) - len(buys) - len(sells)

print(f"   🟢 BUY signals: {len(buys)}")
for b in sorted(buys, key=lambda x: -x['expected_return']):
    print(f"      {b['ticker']}: {b['expected_return']:+.1f}% expected")
    
print(f"   🔴 SELL signals: {len(sells)}")
for s in sorted(sells, key=lambda x: x['expected_return']):
    print(f"      {s['ticker']}: {s['expected_return']:+.1f}% expected")

print(f"   🟡 HOLD signals: {holds}")

In [ ]:
# === CELL 13: SAVE R&D RESULTS ===
results = {
    'generated_at': datetime.now().isoformat(),
    'mode': 'R&D_DISCOVERY',
    'evolution_config': EVOLUTION_CONFIG,
    'generations_run': EVOLUTION_CONFIG['generations'],
    'human_baseline': {
        'params': human_params,
        'test_direction_accuracy': human_acc,
        'test_magnitude_error': human_err,
        'test_buy_avg': human_buy,
        'test_sell_avg': human_sell,
        'fitness': human_fit
    },
    'evolved_best': {
        'params': best_ever.params,
        'test_direction_accuracy': evolved_acc,
        'test_magnitude_error': evolved_err,
        'test_buy_avg': evolved_buy,
        'test_sell_avg': evolved_sell,
        'fitness': evolved_fit
    },
    'improvement': {
        'direction_accuracy_pct': evolved_acc - human_acc,
        'magnitude_error_reduction': human_err - evolved_err,
        'fitness_improvement': evolved_fit - human_fit,
        'verdict': 'EVOLVED_WINS' if evolved_acc > human_acc + 3 else 'MARGINAL' if evolved_acc > human_acc else 'HUMAN_WINS'
    },
    'r_and_d_discoveries': discoveries,
    'evolution_history': history,
    'demo_forecasts': forecasts,
    'experimental_weights': {
        'rsi_divergence': best_ever.params.get('w_rsi_divergence', 0),
        'consolidation': best_ever.params.get('w_consolidation', 0),
        'trend_structure': best_ever.params.get('w_trend_structure', 0),
        'mean_reversion': best_ever.params.get('mean_reversion_weight', 0)
    }
}

with open('forecast_evolution_results.json', 'w') as f:
    json.dump(results, f, indent=2, default=str)

print("✅ R&D Results saved to forecast_evolution_results.json")

# Also save just the optimal parameters for easy import
optimal_params = {
    'source': 'FORECAST_EVOLUTION_R&D',
    'generated_at': datetime.now().isoformat(),
    'direction_accuracy': evolved_acc,
    'params': best_ever.params
}

with open('evolved_forecast_params.json', 'w') as f:
    json.dump(optimal_params, f, indent=2, default=str)

print("✅ Optimal params saved to evolved_forecast_params.json")

try:
    from google.colab import files
    files.download('forecast_evolution_results.json')
    files.download('evolved_forecast_params.json')
    print("📥 Downloads started!")
except:
    print("(Not in Colab - files saved locally)")

In [ ]:
# === CELL 14: FINAL RECOMMENDATIONS ===
print("=" * 70)
print("📋 FINAL FORECASTER RECOMMENDATIONS")
print("=" * 70)

if evolved_acc > human_acc + 5:  # 5% better
    print("\n🧬 USE EVOLVED FORECASTER!")
    print(f"   Direction accuracy improved by {evolved_acc - human_acc:.1f}%")
    rec = best_ever.params
else:
    print("\n⚖️ BLEND EVOLVED + HUMAN PARAMETERS")
    print("   Evolution found minor improvements.")
    rec = best_ever.params

print("\n🎯 OPTIMAL INDICATOR WEIGHTS:")
print("-" * 50)
for name in ['w_rsi', 'w_macd', 'w_ema_trend', 'w_volume', 'w_momentum', 'w_volatility']:
    indicator = name.replace('w_', '').upper()
    val = rec[name]
    importance = 'HIGH' if val > 1.3 else 'MEDIUM' if val > 0.7 else 'LOW'
    print(f"   {indicator:<15} {val:.2f} ({importance})")

print("\n🔧 COPY THESE SETTINGS TO YOUR FORECASTER:")
print("-" * 50)
print(json.dumps(rec, indent=2))

print("\n" + "=" * 70)
print("✅ FORECAST EVOLUTION COMPLETE!")
print("=" * 70)